In [1]:
import pandas as pd
import numpy as np

In [2]:
import requests

def get_weather(latitude, longitude, date, time, variables):
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&hourly={','.join(variables)}&start_date={date}&end_date={date}&timezone=UTC"
    # Make the API request
    response = requests.get(url)
    data = response.json()
    if not data.get("hourly"):
        print(url, data)
        return None
    datetime_str = f"{date}T{time}"
    for i, hourly_time in enumerate(data["hourly"]["time"]):
        if hourly_time == datetime_str:
            # Extract requested variables for the specific time
            return {var: data["hourly"][var][i] for var in variables}
	
    return None  

# Example usage
latitude = -34.92
longitude = 138.6
date = "2017-01-01"
time = "06:00"  
variables = ["temperature_2m", "relative_humidity_2m", "windspeed_10m", "apparent_temperature"]

weather_data = get_weather(latitude, longitude, date, time, variables)
print(weather_data)

{'temperature_2m': 20.1, 'relative_humidity_2m': 62, 'windspeed_10m': 28.7, 'apparent_temperature': 16.8}


In [3]:
matches = pd.read_csv('../data/all_years_nc_tc_elo_aggr_stats/matches.csv')

In [4]:
matches.head()[['Date', 'time', 'latitude', 'longitude']]

,Date,time,latitude,longitude
0,2017-12-31,07:25,-27.468968,153.023499
1,2017-12-31,09:20,-27.468968,153.023499
2,2018-01-01,08:35,18.521374,73.854507
3,2018-01-01,04:00,-27.468968,153.023499
4,2018-01-01,02:10,-27.468968,153.023499


In [5]:
from datetime import datetime, timedelta

def convert_utc_plus1_to_utc(date, time):
    dt = datetime.strptime(f"{date} {time}", "%Y-%m-%d %H:%M")
	
    # Subtract 1 hour to convert from UTC+1 to UTC
    dt_utc = dt - timedelta(hours=1)
    return dt_utc.strftime("%Y-%m-%d"), dt_utc.strftime("%H:%M")

def round_time_to_nearest_hour(time_str):
    time_obj = datetime.strptime(time_str, "%H:%M")

    # Round minutes to the nearest hour
    if time_obj.minute >= 30:  # Round up
        time_obj += timedelta(minutes=60 - time_obj.minute)
    else:  # Round down
        time_obj -= timedelta(minutes=time_obj.minute)

    return time_obj.strftime("%H:00")

In [6]:
# Apply get_weather to each row
def fetch_weather_data(row):
	date_utc, time_utc = convert_utc_plus1_to_utc(row["Date"], row["time"])
	rounded_time_utc = round_time_to_nearest_hour(time_utc)
	return get_weather(row["latitude"], row["longitude"], date_utc, rounded_time_utc, variables)

# For now just 100 rows to see if it works
#matches = matches.head(100)
# 
# weather_data = matches.apply(fetch_weather_data, axis=1)
# weather_df = pd.DataFrame(weather_data.tolist())
# 
# # Merge the weather data back into the original DataFrame
# matches = pd.concat([matches, weather_df], axis=1)
# print(matches[['Date', 'time', 'latitude', 'longitude', 'temperature_2m', "relative_humidity_2m", "windspeed_10m", "apparent_temperature"]])

In [7]:
def process_partial_chunk(df, func, start_index, chunk_size):
    end_index = min(start_index + chunk_size, len(df))  
    chunk = df.iloc[start_index:end_index]
    print(f"Processing rows {start_index} to {end_index}...")
    
    # Apply the function to the chunk
    chunk_results = chunk.apply(func, axis=1)
    chunk_results.to_csv(f"../data/weather_chunks/{start_index}_{end_index}.csv")
    result_df = pd.DataFrame(chunk_results.tolist(), index=chunk.index)
    print(f"results saved to ../data/weather_chunks/{start_index}_{end_index}.csv")
    return result_df

### To odpaliłem u siebie

In [16]:
weather_data_chunk_1 = process_partial_chunk(matches, fetch_weather_data, start_index=0, chunk_size=500)

Processing rows 0 to 500...


In [17]:
weather_data_chunk_2 = process_partial_chunk(matches, fetch_weather_data, start_index=500, chunk_size=500)

Processing rows 500 to 1000...


In [19]:
weather_data_chunk_3 = process_partial_chunk(matches, fetch_weather_data, start_index=1000, chunk_size=500)

Processing rows 1000 to 1500...
results saved to ../data/weather_chunks/{start_index}_{end_index}.csv


In [21]:
weather_data_chunk_4 = process_partial_chunk(matches, fetch_weather_data, start_index=1500, chunk_size=500)

Processing rows 1500 to 2000...
results saved to ../data/weather_chunks/1500_2000.csv


In [22]:
weather_data_chunk_5 = process_partial_chunk(matches, fetch_weather_data, start_index=2000, chunk_size=500)

Processing rows 2000 to 2500...
results saved to ../data/weather_chunks/2000_2500.csv


In [23]:
weather_data_chunk_6 = process_partial_chunk(matches, fetch_weather_data, start_index=2500, chunk_size=500)

Processing rows 2500 to 3000...
results saved to ../data/weather_chunks/2500_3000.csv


In [24]:
weather_data_chunk_7 = process_partial_chunk(matches, fetch_weather_data, start_index=3000, chunk_size=500)

Processing rows 3000 to 3500...
results saved to ../data/weather_chunks/3000_3500.csv


In [25]:
weather_data_chunk_8 = process_partial_chunk(matches, fetch_weather_data, start_index=3500, chunk_size=500)

Processing rows 3500 to 4000...
results saved to ../data/weather_chunks/3500_4000.csv


In [26]:
weather_data_chunk_9= process_partial_chunk(matches, fetch_weather_data, start_index=4000, chunk_size=500)

Processing rows 4000 to 4500...


KeyError: 'hourly'

In [ ]:
weather_data_chunk_10 = process_partial_chunk(matches, fetch_weather_data, start_index=4500, chunk_size=500)

### To odpal u siebie

In [23]:
weather_data_chunk_11 = process_partial_chunk(matches, fetch_weather_data, start_index=5000, chunk_size=500)

Processing rows 5000 to 5500...
results saved to ../data/weather_chunks/5000_5500.csv


### I tak dalej

In [11]:
weather_data_chunk_12 = process_partial_chunk(matches, fetch_weather_data, start_index=5500, chunk_size=500)

Processing rows 5500 to 6000...
results saved to ../data/weather_chunks/5500_6000.csv


In [29]:
weather_data_chunk_13 = process_partial_chunk(matches, fetch_weather_data, start_index=6000, chunk_size=500)

Processing rows 6000 to 6500...
results saved to ../data/weather_chunks/6000_6500.csv


In [31]:
weather_data_chunk_14 = process_partial_chunk(matches, fetch_weather_data, start_index=6500, chunk_size=500)

Processing rows 6500 to 7000...
results saved to ../data/weather_chunks/6500_7000.csv


In [15]:
weather_data_chunk_15 = process_partial_chunk(matches, fetch_weather_data, start_index=7000, chunk_size=500)

Processing rows 7000 to 7500...
results saved to ../data/weather_chunks/7000_7500.csv


In [21]:
weather_data_chunk_16 = process_partial_chunk(matches, fetch_weather_data, start_index=7500, chunk_size=500)

Processing rows 7500 to 8000...


ReadTimeout: HTTPSConnectionPool(host='archive-api.open-meteo.com', port=443): Read timed out. (read timeout=None)

In [ ]:
weather_data_chunk_17 = process_partial_chunk(matches, fetch_weather_data, start_index=8000, chunk_size=500)

In [ ]:
weather_data_chunk_18 = process_partial_chunk(matches, fetch_weather_data, start_index=8500, chunk_size=500)

In [ ]:
weather_data_chunk_19 = process_partial_chunk(matches, fetch_weather_data, start_index=9000, chunk_size=500)

In [ ]:
weather_data_chunk_20 = process_partial_chunk(matches, fetch_weather_data, start_index=9500, chunk_size=500)